# Amazon Product RAG System - Interactive Exploration

This notebook provides an interactive way to explore the RAG system.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import AmazonProductDataLoader
from src.vector_store import VectorStore
from src.llm_handler import MultiLLMManager
from src.rag_system import RAGSystem
from src.evaluation import RAGEvaluator
from src.questions import get_all_questions

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Load Configuration

In [ ]:
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully!")
print(f"Dataset path: {config['dataset']['path']}")
print(f"Embedding model: {config['embedding']['model_name']}")

## 3. Explore Dataset

In [ ]:
# Load dataset
data_loader = AmazonProductDataLoader(config['dataset']['path'])
df = data_loader.load_data()

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

# Display first few rows
df.head()

In [ ]:
# Get statistics
stats = data_loader.get_statistics()

print("Dataset Statistics:")
print(f"Total products: {stats['total_products']}")
print(f"\nMissing values:")
for col, count in stats['missing_values'].items():
    if count > 0:
        print(f"  {col}: {count}")

In [ ]:
# Visualize category distribution if available
if 'category' in df.columns:
    plt.figure(figsize=(12, 6))
    df['category'].value_counts().head(15).plot(kind='barh')
    plt.title('Top 15 Product Categories')
    plt.xlabel('Count')
    plt.ylabel('Category')
    plt.tight_layout()
    plt.show()

## 4. Build Vector Index

In [ ]:
# Preprocess data
df = data_loader.preprocess_data()
documents = data_loader.create_documents()

print(f"Created {len(documents)} documents")
print(f"\nSample document:")
print(documents[0]['text'][:300])

In [ ]:
# Create vector store
vector_store = VectorStore(
    embedding_model_name=config['embedding']['model_name'],
    index_path=config['vector_db']['index_path'],
    use_faiss=True
)

# Create embeddings
embeddings = vector_store.create_embeddings(documents, batch_size=32)
print(f"Embeddings shape: {embeddings.shape}")

In [ ]:
# Build index
vector_store.build_index()
print("Vector index built!")

# Save index
vector_store.save_index()
print("Index saved!")

## 5. Test Retrieval

In [ ]:
# Test query
test_query = "Best wireless headphones with noise cancellation"

results = vector_store.search(test_query, top_k=5)

print(f"Query: {test_query}\n")
print("Top 5 Results:\n")

for i, result in enumerate(results):
    print(f"{i+1}. Score: {result['score']:.3f}")
    print(f"   {result['document']['text'][:200]}...\n")

## 6. Load LLMs (Choose One)

In [ ]:
# Load a single LLM for testing (Phi-3 is smallest/fastest)
llm_configs = {
    'phi3': config['llms']['phi3']
}

llm_manager = MultiLLMManager(llm_configs)
llm_manager.load_llm('phi3')

print("LLM loaded successfully!")

## 7. Create RAG System

In [ ]:
rag_system = RAGSystem(
    vector_store=vector_store,
    llm_manager=llm_manager,
    top_k=config['rag']['top_k'],
    system_prompt=config['prompts']['system_prompt'],
    qa_template=config['prompts']['qa_template']
)

print("RAG system ready!")

## 8. Test RAG System

In [ ]:
# Test question
question = "What are the best budget-friendly smartphones under $300?"

result = rag_system.query(
    query=question,
    llm_name='phi3',
    return_context=True
)

print(f"Question: {question}\n")
print("="*80)
print("Answer:")
print("="*80)
print(result['answer'])

In [ ]:
# View retrieved context
print("Retrieved Documents:\n")
for i, doc in enumerate(result['retrieved_docs']):
    print(f"{i+1}. Score: {doc['score']:.3f}")
    print(f"   {doc['document']['text'][:150]}...\n")

## 9. Evaluate Answer

In [ ]:
evaluator = RAGEvaluator()

metrics = evaluator.evaluate_answer(
    answer=result['answer'],
    context=result['context'],
    query=question
)

print("Evaluation Metrics:\n")
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")

## 10. Test Multiple Questions

In [ ]:
# Get evaluation questions
questions = get_all_questions()

print(f"Total questions: {len(questions)}\n")

# Display questions
for q in questions[:5]:
    print(f"{q['id']}. {q['question']}")
    print(f"   Category: {q['category']} | Difficulty: {q['difficulty']}\n")

In [ ]:
# Test first 3 questions
results_list = []

for q in questions[:3]:
    print(f"\nProcessing Q{q['id']}: {q['question']}")
    
    result = rag_system.query(
        query=q['question'],
        llm_name='phi3'
    )
    
    results_list.append(result)
    
    print(f"Answer: {result['answer'][:150]}...")

print("\nDone!")

## 11. Interactive Query

In [ ]:
# Interactive query function
def ask_question(question):
    result = rag_system.query(
        query=question,
        llm_name='phi3',
        return_context=True
    )
    
    print("="*80)
    print(f"Q: {question}")
    print("="*80)
    print(f"\nA: {result['answer']}\n")
    print("-"*80)
    print(f"Retrieved {result['num_retrieved']} documents")
    print("-"*80)
    
    # Evaluate
    metrics = evaluator.evaluate_answer(
        answer=result['answer'],
        context=result['context'],
        query=question
    )
    
    print("\nMetrics:")
    for k, v in metrics.items():
        print(f"  {k}: {v:.3f}")
    
    return result

# Try it
ask_question("What are the most popular gaming laptops?")

## 12. Visualization

In [ ]:
# If you have results from multiple questions
if results_list:
    # Extract metrics
    lengths = []
    for r in results_list:
        lengths.append(len(r['answer'].split()))
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(lengths)), lengths)
    plt.xlabel('Question ID')
    plt.ylabel('Answer Length (words)')
    plt.title('Answer Lengths Across Questions')
    plt.tight_layout()
    plt.show()

## Next Steps

1. Load all three LLMs for comparison
2. Run evaluation on all questions
3. Compare model performance
4. Generate final report

Use the main.py script for full evaluation:
```bash
python main.py --build-index
```